# Skipped Annotated

# Primitive Wrappers
## LiteralString
- Used for sensitive APIs

## Literal
- Specific primitive value
- Conjunctive feature: Literal[-3, b"foo", MyEnum.A] are all Unions

## Final
- Will change to a Literal
- Not context-sensitive... cant be substituted
- Rules (Only enforced by linters (ex. mypy)) [mypy and pylance differ here]
    1. Final names: Shouldnt be reassigned after init
    2. Final methods: Shouldnt be overriden in a subclass
    3. Final classes: shouldnt be subclassed
    4. There can be at most one final declaration (without immediate init)
    5. Final can only be used in the outermost position/type
    6. Dont use Final and ClassVar together (mypy determines the scope automatically)
- Two ways to define
    - Implicit... ID: Final = 1
    - Explicit... ID: Final[int] = 1

## Intelligent Indexing
- We can use Literal types to more precisely index into structured heterogeneous types such as tuples, NamedTuples, and TypedDicts. This feature is known as intelligent indexing.
- For example, when we index into a tuple using some int, the inferred type is normally the union of the tuple item types. However, if we want just the type corresponding to some particular index, we can use Literal types like so:

## Tagged Unions
- However, it is not always possible or convenient to do this. For example, it is not possible to use isinstance to distinguish between two different TypedDicts since at runtime, your variable will simply be just a dict.
- Instead, what you can do is label or tag your TypedDicts with a distinct Literal type. Then, you can discriminate between each kind of TypedDict by checking the label:

In [2]:
from typing import *

In [ ]:
def run_query(sql: LiteralString) -> None:
    ...

def caller(arbitrary_string: str, literal_string: LiteralString) -> None:
    run_query("SELECT * FROM students")  # OK
    run_query(literal_string)  # OK
    run_query("SELECT * FROM " + literal_string)  # OK
    run_query(arbitrary_string)  # type checker error
    run_query(  # type checker error
        f"SELECT * FROM students WHERE name = {arbitrary_string}"
    )

In [ ]:
@overload
def fetch_data(raw: Literal[True]) -> bytes: ...
@overload
def fetch_data(raw: Literal[False]) -> str: ...


reveal_type(fetch_data(True))        # Revealed type is "bytes"
reveal_type(fetch_data(False))       # Revealed type is "str"
####################################
PrimaryColors = Literal["red", "blue", "yellow"]
SecondaryColors = Literal["purple", "green", "orange"]
AllowedColors = Literal[PrimaryColors, SecondaryColors]

def paint(color: AllowedColors) -> None: ...

paint("red")        # Type checks!
paint("turquoise")  # Does not type check
####################################
def expects_literal(x: Literal[19]) -> None: pass

c: Final = 19 # type: ignore
d: Final[int] = 20

reveal_type(c)          # Revealed type is "Literal[19]?"
expects_literal(c)      # ...and this type checks!
####################################
# Called intelligent indexing
# We can do the same thing with with TypedDict and str keys:
class MyDict(TypedDict):
    name: str
    main_id: int
    backup_id: int

d: MyDict = {"name": "Saanvi", "main_id": 111, "backup_id": 222}
name_key: Final = "name" # type: ignore
reveal_type(d[name_key])  # Revealed type is "str"

# You can also index using unions of literals
id_key: Literal["main_id", "backup_id"]
reveal_type(d[id_key])    # Revealed type is "int"
####################################
PossibleValues = Literal['one', 'two']

def validate(x: PossibleValues) -> bool:
    match x:
        case 'one':
            return True
        case 'two':
            return False
    assert_never(x) # Only needed for dynamic runtime checking

In [ ]:
# Final overriding a RO property
## Not recommended
class Base:
    @property
    def ID(self) -> int: ...

class Derived(Base):
    ID: Final[int] = 1  # OK

In [ ]:
# Final only guarantees that the name wont be shadowed, doesnt make the value immutable
## Use immutable ABCs with Final
x: Final = ['a', 'b']
x.append('c')  # OK

y: Final[Sequence[str]] = ['a', 'b']
y.append('x')  # Error: Sequence is immutable
z: Final[Tuple[str, ...]] = ('a', 'b')  # Also an option

In [ ]:

class Base:
    @final
    def common_name(self) -> None:
        ...

class Derived(Base):
    def common_name(self) -> None:  # Error: cannot override a final method
        ...


In [ ]:
class Base:
    # Overloaded fxns must consist of 2+ implemented variants, and must be adjacenet in code
    @overload
    def method(self) -> None: ...
    @overload
    def method(self, arg: int) -> int: ...
    @final
    def method(self, x=None):
        ...


In [ ]:
@final
class Leaf:
    ...

class MyLeaf(Leaf):  # Error: Leaf can't be subclassed
    ...

# Concatenate
- RETURN after going through ParamSpec generics
- https://docs.python.org/3.14/library/typing.html#module-contents

# Type Narrowing Tools
- Used by static type checkers to determine a more precise type of an expression within a program's code flow
1. TypeIs: annotate the return type of a user-defined predicate function
    - Only accepts a single type arg, and returns a boolean at runtime
    - IMPORTANT: If the return value is True, the type of its argument is the intersection of the argument’s original type and NarrowedType
    - In the if branch (when the function returns True), the type of the argument is narrowed to the intersection of its original type and T.
    - In the else branch (when the function returns False), the type of the argument is narrowed to the intersection of its original type and the complement of T.
2. TypeGuard (smart alias for a bool return type)
    - Same as TypeIs except (IMPORTANT): if the return value is True, the type of its argument is the type inside TypeGuard

- When a TypeGuard function returns True, type checkers narrow the type of the variable to exactly the TypeGuard type. When a TypeIs function returns True, type checkers can infer a more precise type combining the previously known type of the variable with the TypeIs type. (Technically, this is known as an intersection type.)
- TypeIs requires the narrowed type to be a subtype of the input type, while TypeGuard does not. 

In [ ]:
class Parent: pass
class Child(Parent): pass
@final
class Unrelated: pass

def is_parent(val: object) -> TypeIs[Parent]:
    return isinstance(val, Parent)

def run(arg: Child | Unrelated):
    if is_parent(arg):
        # Type of ``arg`` is narrowed to the intersection
        # of ``Parent`` and ``Child``, which is equivalent to
        # ``Child``.
        assert_type(arg, Child)
    else:
        # Type of ``arg`` is narrowed to exclude ``Parent``,
        # so only ``Unrelated`` is left.
        assert_type(arg, Unrelated)

In [ ]:
def is_str_list(val: list[object]) -> TypeGuard[list[str]]:
    '''Determines whether all objects in the list are strings'''
    return all(isinstance(x, str) for x in val)

def func1(val: list[object]):
    if is_str_list(val):
        # Type of ``val`` is narrowed to ``list[str]``.
        print(" ".join(val))
    else:
        # Type of ``val`` remains as ``list[object]``.
        print("Not a list of strings!")

In [ ]:
def is_two_element_tuple[T](val: tuple[T, ...]) -> TypeGuard[tuple[T, T]]:
    return len(val) == 2

def func(names: tuple[str, ...]):
    if is_two_element_tuple(names):
        reveal_type(names)  # tuple[str, str]
    else:
        reveal_type(names)  # tuple[str, ...]

In [ ]:
def is_set_of[T](val: set[Any], type: type[T]) -> TypeGuard[set[T]]:
    return all(isinstance(x, type) for x in val)

items: set[Any]
if is_set_of(items, str):
    reveal_type(items)  # set[str]

In [ ]:
def is_instance_of[T](val: Any, typ: type[T]) -> TypeIs[T]:
    return isinstance(val, typ)

def process(x: Any) -> None:
    if is_instance_of(x, int):
        reveal_type(x)  # Revealed type is 'int'
        print(x + 1)  # ok
    else:
        reveal_type(x)  # Revealed type is 'Any'

# ClassVar
- Should not 
    1. be set on instances of that class
    2. be used with isinstance() or issubclass()
    3. only accept types and cant be further subscribed

In [ ]:
class Starship:
    stats: ClassVar[dict[str, int]] = {} # class variable
    damage: int = 10                     # instance variable



enterprise_d = Starship(3000)
enterprise_d.stats = {} # Error, setting class variable on instance
Starship.stats = {}     # This is OK

# TypedDict
- Doesnt allow arbitrary keys to be added/removed at runtime
- Subtype of Mapping[str, object]
## Special Types
1. Required
2. NotRequired
3. ReadOnly
4. Unpack: two use cases... the important one is annotating the **kwargs of a function

Results
    - Even with total=False, if a superclass has Required or total=True, there will be a linter error
    - Superclasses with total=True can still add NotRequired items at runtime
    - Mypy doesnt catch the ReadOnly annotations, but pyright does
    - For pyright, a ReadOnly annotation must be initialized at creation, else it's an error during runtime (no matter if it's blank to begin with)
    - For pyright, TypedDict attributes have not been updated to Python 3.13 yet

In [ ]:
from datetime import datetime
from typing import TypedDict, ReadOnly, NotRequired, Required, Unpack, assert_type

type Date = datetime

# Class-based definition
class MovieBase(TypedDict):
    name: ReadOnly[str]
    year: int
    rating: NotRequired[float]
    blank: Required[bool]

# Check if this works
class Movie[T](MovieBase, total=False):
    based_on: T
    checked_out: ReadOnly[Date]
    person_check_out_to: ReadOnly[Required[T]] # See if this works! 

m: Movie[str] = {"name": "Jaws", "year": 1981, 'person_check_out_to': 'Sussy', 'blank': False}

m['rating'] = 4.2

# m['name'] = 'Sharks' # Error

# m['checked_out'] = datetime.now() # Error

def check_movie(**kwargs: Unpack[Movie]) -> None:
    assert_type(kwargs, Movie)
    
    m_required = frozenset({'person_check_out_to', 'name', 'year', 'blank'})
    m_optional = frozenset({'based_on', 'checked_out', 'rating'})
    m_mutable = frozenset({'based_on', 'year', 'rating', 'blank'})

    got = (Movie.__required_keys__, Movie.__optional_keys__, Movie.__mutable_keys__)
    expected = (m_required, m_optional, m_mutable)

    for got_keys, expected_keys in zip(got, expected):
        if sorted(list(got_keys)) != sorted(list(expected_keys)):
            print(f'Wrong! {got_keys} {expected_keys}')
            
check_movie(**m)

In [ ]:
Movie = TypedDict('Movie', {'name': str, 'year': int}, total=False)

movie: Movie = {'name': 'Blade Runner', 'year': 1982}

# Can also act as a CTOR
toy_story = Movie(name='Toy Story') 
# Since total=False, if that wasnt true you'd have to add , year=1995)
toy_story['year'] = 1995 # ok since total=False originally

In [ ]:
def print_typed_dict(obj: Mapping[str, object]) -> None:
    for key, value in obj.items():
        print(f'{key}: {value}')

print_typed_dict(Movie(name='Toy Story', year=1995))  # OK

In [ ]:
from datetime import datetime

type Date = datetime

# Class-based definition
class MovieBase(TypedDict):
    name: ReadOnly[str]
    year: int
    rating: NotRequired[float]

# Check if this works
class Movie[T](MovieBase, total=False):
    based_on: T
    checked_out: ReadOnly[Date]
    person_check_out_to: ReadOnly[Required[T]] # See if this works! 

# Test if the MovieBase items are required in the Movie subclass
m: Movie[T: str] = {"name": "Jaws", "year": 1981, 'person_check_out_to': 'Sussy'}
# Check if you can add rating after the fact
m['rating'] = 4.2

m['name'] = 'Sharks' # Error
# Need to check to see if this is allowed for RO vars outside of init
m['checked_out'] = datetime.now()
m['checked_out'] = datetime.now() # Error

m_required = frozenset({'person_check_out_to', 'name', 'year'})
m_optional = frozenset({'based_on', 'checked_out', 'rating'})
m_mutable = frozenset({'based_on', 'year', 'rating'})

got = (m.__required_keys__, m.__optional_keys__, m.__mutable_keys__)
expected = (m_required, m_optional, m_mutable)

for got_keys, expected_keys in zip(got, expected):
    if got_keys != expected_keys:
        print(f'Wrong! {got_keys} {expected_keys}')


# Advanced Self

In [ ]:
class Tag[T]:
    item: T

    def uppercase_item(self: Tag[str]) -> str:
        return self.item.upper()

def label(ti: Tag[int], ts: Tag[str]) -> None:
    ti.uppercase_item()  # E: Invalid self argument "Tag[int]" to attribute function
                         # "uppercase_item" with type "Callable[[Tag[str]], str]"
    ts.uppercase_item()  # This is OK

In [ ]:
from collections.abc import Sequence

class Storage[T]:
    def __init__(self, content: T) -> None:
        self._content = content

    def first_chunk[S](self: Storage[Sequence[S]]) -> S:
        return self._content[0]

page: Storage[list[str]]
page.first_chunk()  # OK, type is "str"

Storage(0).first_chunk()  # Error: Invalid self argument "Storage[int]" to attribute function
                          # "first_chunk" with type "Callable[[Storage[Sequence[S]]], S]"

In [ ]:
class Lockable(Protocol):
    @property
    def lock(self) -> Lock: ...

class AtomicCloseMixin:
    def atomic_close(self: Lockable) -> int:
        with self.lock:
            # perform actions

class AtomicOpenMixin:
    def atomic_open(self: Lockable) -> int:
        with self.lock:
            # perform actions

class File(AtomicCloseMixin, AtomicOpenMixin):
    def __init__(self) -> None:
        self.lock = Lock()

class Bad(AtomicCloseMixin):
    pass

f = File()
b: Bad
f.atomic_close()  # OK
b.atomic_close()  # Error: Invalid self type for "atomic_close"

# ParamSpec
- Parameter specification variable. A specialized version of type variables
- Used to forward the parameter types of one callable to another callable
- Only valid with
    1. Concatenate
    2. As the first arg to a Callable
    3. As parameters for user-defined Generics
- ParamSpec captures both positional and keyword parameters
    - P.args (instance of ParamSpecArgs)
    - P.kwargs (instance of ParamSpecKwargs)

In [ ]:
from collections.abc import Callable
import logging

def add_logging[T, **P](f: Callable[P, T]) -> Callable[P, T]:
    '''A type-safe decorator to add logging to a function.'''
    def inner(*args: P.args, **kwargs: P.kwargs) -> T:
        logging.info(f'{f.__name__} was called')
        return f(*args, **kwargs)
    return inner

@add_logging
def add_two(x: float, y: float) -> float:
    '''Add two numbers together.'''
    return x + y

In [ ]:
type IntFunc[**P] = Callable[P, int]

In [ ]:
cb3: Callable[Concatenate[int, ...], str]
cb3 = lambda x: str(x)  # OK
cb3 = lambda a, b, c: str(a)  # OK
cb3 = lambda : ""  # Error
cb3 = lambda *, a: str(a)  # Error

In [ ]:
type Callback[**P] = Callable[P, str]

def func(cb: Callable[[], str]) -> None:
    f: Callback[...] = cb  # OK

# Metaprogramming with Python

# Have ChatGPT show specific GOF Patterns using paramspec and traits bounds

# Python-course.eu
- Chapters: 16-18

# Ch. 16: Dynamically Creating Classes with Type
- A user-defined class (or the class "object") is an instance of the class "type". So, we can see, that classes are created from type. In Python3 there is no difference between "classes" and "types". They are in most cases used as synonyms.
- We can create classes, which inherit from the class "type". So, a metaclass is a subclass of the class "type".
- Type can be called with three parameters
    - $type(classname, superclasses, attributes\_dict)$
    - $type(str, tuple | list, dict)$
    - Returns a new type object
- When we call "type", the call method of type is called. The call method runs two other methods: new and init:
    - type.\__new__(typeclass, classname, superclasses, attributedict)
    - type.\__init__(cls, classname, superclasses, attributedict)


In [1]:
A = type("A", (), {})
x = A()
print(type(x))

<class '__main__.A'>


In [2]:
class Robot:

    counter = 0

    def __init__(self, name):
        self.name = name

    def sayHello(self):
        return "Hi, I am " + self.name


def Rob_init(self, name):
    self.name = name

Robot2 = type("Robot2", 
              (), 
              {"counter":0, 
               "__init__": Rob_init,
               "sayHello": lambda self: "Hi, I am " + self.name})

x = Robot2("Marvin")
print(x.name)
print(x.sayHello())

y = Robot("Marvin")
print(y.name)
print(y.sayHello())

print(x.__dict__)
print(y.__dict__)

Marvin
Hi, I am Marvin
Marvin
Hi, I am Marvin
{'name': 'Marvin'}
{'name': 'Marvin'}


## Metaclasses
- A metaclass is a class whose instances are classes: control the creation and structure of clases
    - Often override \__new__ and \__init__

### Fix/Modify all of the code below

In [3]:
import re

# Custom Metaclass Definition
class CamelCaseToUnderscoreMeta(type):
    def __new__(cls, name, bases, dct):
        # Create a separate dictionary for modified items
        modified_items = {}

        # Convert camel case method names to underscore notation
        for key, value in dct.items():
            if callable(value) and re.match(r'^[a-z]+(?:[A-Z][a-z]*)*$', key):
                # Convert camel case to underscore notation
                underscore_name = re.sub(r'([a-z0-9])([A-Z])', r'\1_\2', key).lower()
                # Store the modified item in the new dictionary
                modified_items[underscore_name] = value
            else:
                # Store unmodified items in the new dictionary
                modified_items[key] = value

        # Create the class using the modified attributes
        return super().__new__(cls, name, bases, modified_items)

# Class using the Custom Metaclass
class CamelCaseClass(metaclass=CamelCaseToUnderscoreMeta):
    def processData(self):
        print("Processing data...")

    def transformData(self):
        print("Transforming data...")

    def processOutputData(self):
        print("Processing output data...")

# Creating an instance of CamelCaseClass
camel_case_instance = CamelCaseClass()

# Calling methods with modified underscore notation names
camel_case_instance.process_data()
camel_case_instance.transform_data()
camel_case_instance.process_output_data()

Processing data...
Transforming data...
Processing output data...


In [4]:
class FuncCallCounter(type):
    """ A Metaclass which decorates all the methods of the 
        subclass using call_counter as the decorator
    """
    
    @staticmethod
    def call_counter(func):
        """ Decorator for counting the number of function 
            or method calls to the function or method func
        """
        def helper(*args, **kwargs):
            helper.calls += 1
            return func(*args, **kwargs)
        helper.calls = 0
        helper.__name__= func.__name__
    
        return helper
    
    
    def __new__(cls, clsname, superclasses, attributedict):
        """ Every method gets decorated with the decorator call_counter,
            which will do the actual call counting
        """
        for attr in attributedict:
            if callable(attributedict[attr]) and not attr.startswith("__"):
                attributedict[attr] = cls.call_counter(attributedict[attr])
        
        return type.__new__(cls, clsname, superclasses, attributedict)
    

class A(metaclass=FuncCallCounter):
    
    def foo(self):
        pass
    
    def bar(self):
        pass

if __name__ == "__main__":
    x = A()
    print(x.foo.calls, x.bar.calls)
    x.foo()
    print(x.foo.calls, x.bar.calls)
    x.foo()
    x.bar()
    print(x.foo.calls, x.bar.calls)

0 0
1 0
2 1


In [ ]:
from typing import Generic
from threading import Lock

class SynchronizedMethod[**P, T]:
    """Descriptor that applies a lock to instance methods."""
    _lock: ClassVar[Lock] = Lock()

    def __init__(self, method: Callable[P, T]) -> None:
        self.method = method

    def __get__(self, instance: object, owner: type | None = None) -> Callable[P, T]:
        if instance is None:
            return self.method  # Allow access to original method from class

        def wrapper(*args: P.args, **kwargs: P.kwargs) -> T:
            with self._lock:
                return self.method(instance, *args, **kwargs)
        return wrapper

class LockedCounter:
    def __init__(self) -> None:
        self._count = 0

    @SynchronizedMethod
    def increment(self) -> None:
        self._count += 1

    @SynchronizedMethod
    def get_count(self) -> int:
        return self._count


In [ ]:
from typing import Any, Type, Callable, ParamSpec, TypeVar, ClassVar
from threading import Lock

class SynchronizedMeta[**P, T]:
    """Metaclass that applies locking to all instance methods."""
    _lock: ClassVar[Lock] = Lock()

    def __new__(cls: Type[type], name: str, bases: tuple, dct: dict) -> Type[Any]:
        for attr_name, attr_value in dct.items():
            if callable(attr_value) and not attr_name.startswith("_"):
                dct[attr_name] = cls._wrap_with_lock(attr_value)
        return super().__new__(cls, name, bases, dct)

    @classmethod
    def _wrap_with_lock(cls, method: Callable[P, T]) -> Callable[P, T]:
        """Wraps instance methods with a lock."""
        def wrapper(self: object, *args: P.args, **kwargs: P.kwargs) -> T:
            with cls._lock:
                return method(self, *args, **kwargs)
        return wrapper

class AutoLocked(metaclass=SynchronizedMeta):
    """All methods in this class are synchronized automatically."""
    def __init__(self) -> None:
        self._value = 0

    def increment(self) -> None:
        self._value += 1

    def get_value(self) -> int:
        return self._value